# Projeto Novelas do Brasil



A ideia desse projeto veio da necessidade criar um portifólio para colocar a prova todo o conhecimento que venho adquirindo com os meus estudos na área de dados, foi onde analisando oportunidades de emprego no LinkedIn  que me deparei com a seguinte vaga Analista de dados / Ciência de dados júnior na rede Globo para audiência, foi olhando essa vaga de emprego que me desafiei em criar um dashboard sobre a audiência das novelas no Brasil.

# 1° Etapa – Extrair / Web Scraping

A primeira dificuldade encontrar os dados históricos de audiência de cada novela, primeiro lugar que procurei foi no site do ibope, mas não conta com esses dados a disposição, não pelo menos de forma gratuita.


Procurando na internet encontrei o seguinte site http://gabrielfarac.blogspot.com/p/audiencia-detalhada.html , onde possui uma série histórica  de audiência das principais emissoras e suas novelas e programas entretanto esse site no possui todos os dados que preciso sobre as novelas então para complementar vamos raspar os dados do site http://teledramaturgia.com.br/ , como os dados não estão estruturados em uma tabela, banco de dados etc. 

Vamos precisar extrair os dados desse site utilizando técnicas de web scraping.

Ferramentas Utilizadas:
    Python
    Biblioteca Beautifulsoup
    Biblioteca Pandas
    Vamos entrar em pagina por pagina para extrair os dados de cada novela.


In [1]:
!pip install unidecode

In [2]:
# Importando bibliotecas
from unidecode import unidecode
from urllib.request import urlopen, urlretrieve, Request
from bs4 import BeautifulSoup
import pandas as pd
import math
import requests
from urllib.error import HTTPError


In [3]:
def dados_site(site): # Função para abrir o site e carregar no beautifulsoup

    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        req = Request(site, headers = headers)
        response = urlopen(req)    
    except:
        return "404 Not Found"

    html = response.read().decode('utf-8')
    html_soup = BeautifulSoup(html, 'html.parser')
    return html_soup

## Extração Site [Gabriel Farac](http://gabrielfarac.blogspot.com/p/audiencia-detalhada.html)

In [13]:
## Vamos pegar os links  
url = 'http://gabrielfarac.blogspot.com/p/audiencia-detalhada.html' #Site onde vamos fazer a raspagem de dados
links_site_audiencia = dados_site(url) #Chamamos a função para abrir e retorna o site
links_site_audiencia = links_site_audiencia.find('tbody') #Aqui vamos pegar a tabela com todos os programas
links_site_audiencia = links_site_audiencia.find_all('a') #Vamos localizar todos os links
len(links_site_audiencia)
links_audiencias = [] #Todos os links para os subsites das novelas
for link in links_site_audiencia: #Aqui vamos percorrer todos os link e adicionar a uma lista esses link
  
  links_audiencias.append(link.get('href'))


del links_audiencias[0]

### Extração Média

In [8]:
def emissora_horario (emissora_horario): #Função para localizar e separar a emissora da horario
  emissora_horario = emissora_horario.replace('\n','')
  emissora_horario = emissora_horario.split('– ')
  if len(emissora_horario)<2:
    return "erro","erro"
  return emissora_horario[0], emissora_horario[1]

dnovela_media = pd.DataFrame()
erro =[] #Aqui vamos armazenar os link que tiveram algum erro
for link in links_audiencias: # vamos acessar cada link e pegar a tabela daquele link e concatenar com o dataframe dnovela_media
  try:
    site_novelas = dados_site(link)
    tabela_novela = pd.read_html(link)
    tabela_novela = tabela_novela[0]
    emissora, horario = emissora_horario(site_novelas.find('h3', {'class':'post-title entry-title'}).text)
    tabela_novela['Emissora'] = emissora
    tabela_novela['Horario'] = horario
    dnovela_media = pd.concat([dnovela_media, tabela_novela])
  except:
    erro.append(link)
    continue


In [171]:
dnovela_media.to_csv('dnovela_media.csv') # Vamos salver esse dataframe em um arquivo csv
dnovela_media

,0,1,2,Emissora,Horario
0,Ano,Novela,Média,GLOBO,EDIÇÃO ESPECIAL
1,2021,O Cravo e a Rosa,140,GLOBO,EDIÇÃO ESPECIAL
2,2022,Chocolate com Pimenta,-,GLOBO,EDIÇÃO ESPECIAL
0,Ano,Novela,Média,GLOBO,VALE A PENA VER DE NOVO
1,1980,Dona Xepa,-,GLOBO,VALE A PENA VER DE NOVO
...,...,...,...,...,...
7,REPRISES,REPRISES,NaN,REDETV!,NOVELAS
8,NaN,NaN,NaN,REDETV!,NOVELAS
9,Ano,Novela,Média,REDETV!,NOVELAS
10,2004,"Betty, a Feia",11,REDETV!,NOVELAS


### Extração Completa

In [11]:
def achar_audiencia(lista_site,ano,emissora,horario,novela): #Função para extrair cada audiencia linha por linha
    audiencia = []
    media_geral = '0'
    dia_da_semana = ['Segunda','Terça','Quarta','Quinta','Sexta','Sabado','Domingo']
    autor = ""
    autor_sim_nao = 0
    capitulo = 1
    for linha in lista_site:
        if autor_sim_nao != 0:
            autor = linha.text
            autor_sim_nao = 0
        if linha.text.find('uma novela de' or "novela de")!= -1:
            autor_sim_nao = 1

        if linha.text.find('Semana ')!= -1:  ##
            semana = linha.text
            semana = semana.split(':')
            semana = semana
        if linha.text.find(' = ')!= -1:
            audiencia_dias = linha.text
            audiencia_dias = audiencia_dias.replace('\n','')
            audiencia_dias = audiencia_dias.replace('\xa0','')
            audiencia_dias = audiencia_dias.split('=')
            Data = audiencia_dias[0]
            audiencia_dias = audiencia_dias[1].split('|')

            cont_semana = 0
            for audiencia_dia in audiencia_dias:
                audiencia_semanal ={}
                audi = audiencia_dia
                audiencia_semanal['Novela'] = novela
                audiencia_semanal['Autor'] = autor
                audiencia_semanal['Semana'] = semana[0]
                audiencia_semanal['Dia Semana'] = dia_da_semana[cont_semana]
                audiencia_semanal['Data'] = Data
                audiencia_semanal['Pontos Audiencia'] = audi
                audiencia_semanal['ano'] = ano
                audiencia_semanal['Emissora'] = emissora
                audiencia_semanal['horario'] = horario
                audiencia_semanal['Capitulo'] = capitulo
                capitulo += 1
                cont_semana += 1 

                audiencia.append(audiencia_semanal)


        if linha.text.find('MÉDIA GERAL: ')!= -1:
            media_geral = linha.text
            media_geral= media_geral.replace('MÉDIA GERAL: ','')
            media_geral= media_geral.replace('.',',')



    return audiencia , media_geral

def emissora_horario (emissora_horario): # separa emissora de horario
  emissora_horario = emissora_horario.replace('\n','')
  emissora_horario = emissora_horario.split('– ')
  if len(emissora_horario)<2:
    return "erro","erro"
  return emissora_horario[0], emissora_horario[1]

def novela_ano(novela):#Separa o nome da novela do ano
  novela_ano = novela.replace('\n','')
  novela_ano = novela.replace(')','')
  novela_ano = novela.split('(')
  if len(novela_ano) < 2:
    novela ="Erro"
    ano = "Erro"
    return novela , ano
  novela = novela_ano[0]
  ano = novela_ano[-1]
  return novela , ano


In [14]:
audiencia_novelas_dia =[] #Lista de audiencia de novelas por dia
erro = [] #Lista de erro da raspagem
for link_programacao in links_audiencias: #Vamos percorrer cada de link de cada programação
  link_site_audiencia = dados_site(link_programacao) #Vamos pegar os dados do site usando a nossa função
  if link_site_audiencia == "404 Not Found": #Caso não ache o site por algum motivo ele continua o loop
    continue
  novela = {} #Vamos criar um dicinario para adicionar cada linha de 

  Emissora , Horario = emissora_horario(link_site_audiencia.find('h3', {'class':'post-title entry-title'}).text)#Vamos localizar e separar a emissora e horario
  links_novelas = link_site_audiencia.find('tbody').find_all('a') #Pega o link de cada novela
  for link_novela in links_novelas:#Pecorre cada link de cada novela
    try:
        url = link_novela.get('href')#Pega o link da novela
        print(url)

        novela_site = dados_site(url) #Pega os dados da novela
        if novela_site == "404 Not Found":
          continue

        semanas = novela_site.find_all('b')#localiza todo
        novela , ano = novela_ano(novela_site.find('h3').text)
    

        audiencia, media_geral = achar_audiencia(semanas,ano,Emissora,Horario,novela)
    except:
      erro.append(link_programacao)
      continue
    audiencia_novelas_dia.extend(audiencia)
    print(url)


https://gabrielfarac.blogspot.com/2021/12/o-cravo-e-rosa-audiencia-detalhada.html
https://gabrielfarac.blogspot.com/2021/12/o-cravo-e-rosa-audiencia-detalhada.html
https://gabrielfarac.blogspot.com/2022/09/chocolate-com-pimenta-audiencia.html
https://gabrielfarac.blogspot.com/2022/09/chocolate-com-pimenta-audiencia.html
https://gabrielfarac.blogspot.com.br/1981/05/a-sucessora-audiencia-detalhada.html
https://gabrielfarac.blogspot.com.br/1981/05/a-sucessora-audiencia-detalhada.html
https://gabrielfarac.blogspot.com.br/1981/11/te-contei-audiencia-detalhada.html
https://gabrielfarac.blogspot.com.br/1981/11/te-contei-audiencia-detalhada.html
https://gabrielfarac.blogspot.com.br/1982/03/cabocla-audiencia-detalhada.html
https://gabrielfarac.blogspot.com.br/1982/03/cabocla-audiencia-detalhada.html
https://gabrielfarac.blogspot.com.br/1982/08/marron-glace-audiencia-detalhada.html
https://gabrielfarac.blogspot.com.br/1982/08/marron-glace-audiencia-detalhada.html
https://gabrielfarac.blogspot.co

In [15]:
dnovelas_final = pd.DataFrame(audiencia_novelas_dia)
dnovelas_final.to_csv('dnovelas_final.csv')
dnovelas_final

,Novela,Autor,Semana,Dia Semana,Data,Pontos Audiencia,ano,Emissora,horario,Capitulo
0,\nO Cravo e a Rosa,,Semana 02,Segunda,Semana 02: 13/12 a 17/12/2021,10,2021)\n,GLOBO,EDIÇÃO ESPECIAL,1
1,\nO Cravo e a Rosa,,Semana 02,Terça,Semana 02: 13/12 a 17/12/2021,12,2021)\n,GLOBO,EDIÇÃO ESPECIAL,2
2,\nO Cravo e a Rosa,,Semana 02,Quarta,Semana 02: 13/12 a 17/12/2021,12,2021)\n,GLOBO,EDIÇÃO ESPECIAL,3
3,\nO Cravo e a Rosa,,Semana 02,Quinta,Semana 02: 13/12 a 17/12/2021,10,2021)\n,GLOBO,EDIÇÃO ESPECIAL,4
4,\nO Cravo e a Rosa,,Semana 02,Sexta,Semana 02: 13/12 a 17/12/2021,10,2021)\n,GLOBO,EDIÇÃO ESPECIAL,5
...,...,...,...,...,...,...,...,...,...,...
240499,"\nBetty, a Feia",FERNANDO GAITÁN,Semana 40,Segunda,Semana 40: 02/12 a 06/12/2013,01,2013)\n,REDETV!,NOVELAS,196
240500,"\nBetty, a Feia",FERNANDO GAITÁN,Semana 40,Terça,Semana 40: 02/12 a 06/12/2013,01,2013)\n,REDETV!,NOVELAS,197
240501,"\nBetty, a Feia",FERNANDO GAITÁN,Semana 40,Quarta,Semana 40: 02/12 a 06/12/2013,01,2013)\n,REDETV!,NOVELAS,198
240502,"\nBetty, a Feia",FERNANDO GAITÁN,Semana 40,Quinta,Semana 40: 02/12 a 06/12/2013,01,2013)\n,REDETV!,NOVELAS,199


## Extração Site [Teletraumaturgia](http://teledramaturgia.com.br/)

In [5]:
url = 'http://teledramaturgia.com.br/novelas/novelas-ordem-cronologica/' #Site onde vamos fazer a raspagem de dados
site_de_novelas = dados_site(url) #Chamamos a função para abrir e retorna o site

In [6]:
dnovelas_teledra =[]
for ano in site_de_novelas.find_all('div',{"class":'fusion-panel panel-default'}):
    for link in ano.find_all('a'):
        novela = {}
        novela['Nome'] = link.text
        url = str(link.get('href'))
        
        if url.find('http')== -1:
            continue

        novela["link"] = link.get('href')
        novela['Ano']  = ano.find('div',{"class": 'fusion-toggle-heading'}).text
        dnovelas_teledra.append(novela)



In [7]:
url = 'http://teledramaturgia.com.br/uma-rosa-com-amor-2010/'
site = dados_site(url)
teste = site.find_all('div',{"class":"tab-pane"})  
teste2 = site.find('div',{"class":"nav"})
teste2 = teste2.find_all('li')
testex = {}
#for x in range(len(teste2)-1):
#    testex[teste2[x].text] = teste[x].text
teste[0].text, teste2[0].text





('\n\nCARLA MARINS – Serafina Rosa PetroneCLÁUDIO LINS – Claude Antoine GeraldyMÔNICA CARVALHO – Nara Paranhos de VasconcellosEDNEY GIOVENAZZI – Giovanni PetroneBETTY FARIA – AmáliaTONI GARRIDO – FrazãoISADORA RIBEIRO – Roberta VermontJOÃO ACAIABE – PimpinoniFELIPE LIMA – MiltonSABRINA PETRAGLIA – TerezinhaFÁBIO RHODEN – Beto / BentinhoANDRÉ CURSINO – SérgioLÚCIA ALVES – JoanaNILTON BICUDO – AfrânioJUSSARA FREIRE – PepaCARLO BRIANI – Egídio / OlegárioMARINA STACCIARINI – RaquelRENATO SCARPIN – FreitasJOANA LIMAVERDE – JaneteETTY FRASER – AntonietaCLARISSE ABUJAMRA – Catarina Rosa BatateiraRUBENS CARIBÉ – Antoninho / HugoGRETA ANTOINE – CleideLUI STRASSBURGER – OscarCIDINHA MILAN – GenovevaPATHY DEJESUS – AlabáLUCIANA VENDRAMINI – NinicaGISELE FRAGA – AlziraANA CAROLINA DIAS – ErcyJOÃO VITTI – CarlosCLÉO VENTURA – MartaDANIEL UEMURA – GurgelVANESSA KSEIB – SilvinhaROBERTO ARDUIN – Mr. SmithMARIA CLÁUDIA – Mrs. Elizabeth SmithBRUNO BEZERRA – DinoELAM LIMA – ColibriMILA RIBEIRO – DádiMÁRC

In [17]:
def achar_Tema_de_abertura(lista_site):

    for linha in lista_site:
        if linha.text.find('Tema de Abertura:')!= -1:
            musica_abertura = linha.text
            musica_abertura = musica_abertura.replace('Tema de Abertura: ','')
            return musica_abertura
    
def canal_e_horario(op_hr):
    op_hr = op_hr.split('\n')
    op_hr = op_hr[0]
    op_hr = op_hr.split('–')
    canal = op_hr[0]
    horario = op_hr[-1]
    horario = horario[:4]

    return canal,horario

dados_novelas = []
erro_tele = []

for novela_extraida in dnovelas_teledra:
    dado_novela = {}
    nome_novela = novela_extraida['Nome']
    url = novela_extraida['link']

    print(url)
    dados_novela = dados_site(url)
    if dados_novela == '404 Not Found':
        continue
    try:
        novela_abas = dados_novela.find('div',{"class":"nav"})
        novela_abas = novela_abas.find_all('li')

        novela_conteudo_abas = dados_novela.find_all('div',{"class":"tab-pane"})

        dado_novela['Nome'] = novela_extraida['Nome']
        dado_novela['Site'] = url
        dado_novela['Sinopse'] = dados_novela.find('div',{"class":"fusion-column-wrapper"}).text
        dado_novela['Canal'],dado_novela['Horario'] = canal_e_horario(dados_novela.find('div',{"class":"content-container"}).findAll('p')[0].text)
        dado_novela['Capitulos'] = dados_novela.find('div',{"class":"content-container"}).findAll('p')[0].text[-13:]
        dado_novela['imagem'] = dados_novela.find('span',{'class':'fusion-imageframe imageframe-none imageframe-1'}).find('img').get('src')
        dado_novela['Ano'] =  novela_extraida['Ano']
        for x in range(len(novela_abas)):
            dado_novela[novela_abas[x].text] = novela_conteudo_abas[x].text
        dado_novela['Abertura'] = achar_Tema_de_abertura(dados_novela.find('div',{"class":"tab-content"}).findAll('p'))

        print(dado_novela['imagem'])
    except:
        erro_tele.append(url)
        continue
    dados_novelas.append(dado_novela)  
    

    

    


http://teledramaturgia.com.br/tempos-modernos/
http://teledramaturgia.com.br/wp-content/uploads/2015/02/temposmodernos2-300x231.jpg
http://teledramaturgia.com.br/uma-rosa-com-amor-2010/
http://www.teledramaturgia.com.br/wp-content/uploads/2015/02/umarosacomamor2010_logo-300x231.jpg
http://teledramaturgia.com.br/escrito-nas-estrelas/
http://teledramaturgia.com.br/wp-content/uploads/2015/02/escritonasestrelas-300x231.jpg
http://teledramaturgia.com.br/passione/
http://www.teledramaturgia.com.br/wp-content/uploads/2015/02/passione_logo-300x231.jpg
http://teledramaturgia.com.br/ribeirao-do-tempo/
http://www.teledramaturgia.com.br/wp-content/uploads/2015/02/ribeiraodotempo-300x231.jpg
http://teledramaturgia.com.br/ti-ti-ti-2010/
http://teledramaturgia.com.br/wp-content/uploads/2015/02/tititi2010-300x231.jpg
http://teledramaturgia.com.br/araguaia/
http://www.teledramaturgia.com.br/wp-content/uploads/2015/02/araguaia_logo-300x231.jpg
http://teledramaturgia.com.br/insensato-coracao/
http://tele

In [18]:
dnovelas_Teletrau = pd.DataFrame(dados_novelas)
dnovelas_Teletrau.to_csv('dnovelas_Teletrau.csv')


In [19]:
dnovelas_Teletrau = pd.DataFrame(dados_novelas)
dnovelas_Teletrau

,Nome,Site,Sinopse,Canal,Horario,Capitulos,imagem,Ano,Elenco,Núcleos,Bastidores,Trilha Sonora,Abertura,Trilha sonora,Trama Policial,Resumos,Trama dos assassinatos
0,tempos modernos,http://teledramaturgia.com.br/tempos-modernos/,Leal Cordeiro (Antônio Fagundes) é um homem de...,Globo,19h,161 capítulos,http://teledramaturgia.com.br/wp-content/uploa...,2010,\nANTÔNIO FAGUNDES – Leal Cordeiro\nELIANE GIA...,\n– núcleo de LEAL CORDEIRO (Antônio Fagundes)...,\nÚnica novela de Bosco Brasil como autor titu...,\nTrilha Sonora Nacional\n\n01. 10 MINUTOS – A...,CÉREBRO ELETRÔNICO – Myllena,NaN,NaN,NaN,NaN
1,uma rosa com amor,http://teledramaturgia.com.br/uma-rosa-com-amo...,Serafina Rosa Petrone (Carla Marins) é uma sec...,SBT,20h,145 capítulos,http://www.teledramaturgia.com.br/wp-content/u...,2010,\n\nCARLA MARINS – Serafina Rosa PetroneCLÁUDI...,"\n\n– núcleo de SERAFINA ROSA (Carla Marins), ...","\n\nEm 2010, o SBT comprou os direitos da obra...",\n\nTrilha Sonora\n01. PRETTY WOMAN – Roy Orbi...,PRETTY WOMAN – Roy Orbinson,NaN,NaN,NaN,NaN
2,escrito nas estrelas,http://teledramaturgia.com.br/escrito-nas-estr...,"O Dr. Ricardo Aguillar (Humberto Martins), viú...",Globo,18h,143 capítulos,http://teledramaturgia.com.br/wp-content/uploa...,2010,\n\nHUMBERTO MARTINS – Ricardo Aguillar / Pedr...,"\n\n– núcleo de RICARDO (Humberto Martins), vi...","\n\nCom poucos tropeços e um saldo positivo, a...",\n\n\n01. QUANDO A CHUVA PASSAR – Paula Fernan...,QUANDO A CHUVA PASSAR – Paula Fernandes,NaN,NaN,NaN,NaN
3,passione,http://teledramaturgia.com.br/passione/,Quando Bete Gouveia (Fernanda Montenegro) conh...,Globo,21h,209 capítulos,http://www.teledramaturgia.com.br/wp-content/u...,2010,\n\nTONY RAMOS – Totó (Antonio Mattoli)MARIANA...,\n\n– núcleo de BETE GOUVEIA (Fernanda Montene...,"\n\nPassione explorou alguns temas difíceis, c...",\n\nTrilha Sonora Nacional\n\n01. AQUILO QUE D...,AQUILO QUE DÁ NO CORAÇÃO – Lenine,NaN,NaN,NaN,NaN
4,ribeirão do tempo,http://teledramaturgia.com.br/ribeirao-do-tempo/,O jovem empresário Tito (Ângelo Paes Leme) é n...,,,250 capítulos,http://www.teledramaturgia.com.br/wp-content/u...,2010,\n\nBIANCA RINALDI – ArmindaCAIO JUNQUEIRA – J...,"\n\n– núcleo de ARMINDA (Bianca Rinaldi), pode...","\n\nPara escrever Ribeirão do Tempo, o autor, ...",\n\nTrilha Sonora\n01. FÃ – Christian & Cristi...,PIQUE DO TEMPO – Tom Zé,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,dez vidas,http://teledramaturgia.com.br/dez-vidas/,"A Inconfidência Mineira é retratada, tendo com...",,,neiro de 1970,http://www.teledramaturgia.com.br/wp-content/u...,1969,\nCARLOS ZARA – Tiradentes (Joaquim José da Si...,NaN,\nÚltima das treze novelas consecutivas que Iv...,\nTrilha Sonora: Manuel Marques\n\nTEMA DE TIR...,None,NaN,NaN,NaN,NaN
695,seu único pecado,http://teledramaturgia.com.br/seu-unico-pecado/,Um jovem vem para São Paulo à procura de seu t...,Record,19h,embro de 1969,http://www.teledramaturgia.com.br/wp-content/u...,1969,\nADEMIR ROCHA\nSUSANA VIEIRA\nÍRIS BRUZZI\nRO...,NaN,\nPercebe-se na trama a inspiração no livro de...,NaN,None,NaN,NaN,NaN,NaN
696,véu de noiva,http://teledramaturgia.com.br/veu-de-noiva/,"Andreia, moça simples e generosa, de família h...",Globo,20h,21 capítulos,http://teledramaturgia.com.br/wp-content/uploa...,1969,\n\nREGINA DUARTE – Andreia Madeira / Roberta ...,"\n\n– núcleo de ANDREIA (Regina Duarte), jovem...",\nInfluenciada pelo sucesso da novela Beto Roc...,\n\nTrilha Sonora\n\n01. TEMA DE LUCIANO – Lui...,None,NaN,NaN,NaN,NaN
697,super plá,http://teledramaturgia.com.br/super-pla/,"Plácido, quando criança, era super inteligente...",,,maio de 1970,http://www.teledramaturgia.com.br/wp-content/u...,1969,\nRODRIGO SANTIAGO – Plácido\nBETE MENDES – Ti...,NaN,"\nApós o sucesso de Beto Rockfeller, o autor B...",\nTrilha Sonora 1\n\n01. JAM UP JELLY TIGHT – ...,None,NaN,NaN,NaN,NaN


### Extração Autor

In [76]:
url = 'http://teledramaturgia.com.br/menu-de-profissionais/' #Site onde vamos fazer a raspagem de dados
site_de_autor = dados_site(url) #Chamamos a função para abrir e retorna o site

autores=[]
for letra in site_de_autor.find_all('div',{"fusion-one-fourth fusion-layout-column fusion-spacing-yes"}):
    for autor in letra.find_all('a'):
        dautor = {}
        dautor['Autor'] = autor.text
        url = str(autor.get('href'))
        
        if url.find('http')== -1:
            continue

        dautor["link"] = autor.get('href')
        autores.append(dautor)


In [163]:
def Lista_de_novelas_feitas(lista_novelas):
    novelas_feitas = []
    for linha in lista_novelas:
        novelas_feitas.append(linha.text)
    return novelas_feitas

### 
# Como o primeiro site não tinha todas as informações que eu gostaria no dataset, achei as informações em um outro site onde vamos
#fazer a raspagem para adicionar tambem ao nosso dataset
# ###
dautores_teledra = []
erro = []
for autor in autores:
    

    url = autor['link']

    print(url)
    site_autor = dados_site(url)
    if site_autor == '404 Not Found':
        continue

    try:
      for novela in  site_autor.findAll('h4', {"class":'entry-title'}):
        dado_autor = {}
        print(novela)
        dado_autor["Autor"] = autor['Autor']
        dado_autor["Biografia"] = site_autor.find('div', {"class":'fusion-three-fifth fusion-layout-column fusion-column-last fusion-spacing-yes'}).text
        dado_autor["Novelas"] = novela.text
        dado_autor["link"] = novela.find('a').get('href')
        dado_autor["Foto"] = site_autor.find('span',{'class':'fusion-imageframe imageframe-none imageframe-1'}).find('img').get('src')
        dautores_teledra.append(dado_autor)
        print(dado_autor["link"])
    except:
     print('erro')
     erro.append(url)


    

    

http://teledramaturgia.com.br/aguinaldo_silva/
<h4 class="entry-title"><a href="http://teledramaturgia.com.br/plantao-de-policia/">Plantão de Polícia</a></h4>
http://teledramaturgia.com.br/plantao-de-policia/
<h4 class="entry-title"><a href="http://teledramaturgia.com.br/lampiao-e-maria-bonita/">Lampião e Maria Bonita</a></h4>
http://teledramaturgia.com.br/lampiao-e-maria-bonita/
<h4 class="entry-title"><a href="http://teledramaturgia.com.br/bandidos-da-falange/">Bandidos da Falange</a></h4>
http://teledramaturgia.com.br/bandidos-da-falange/
<h4 class="entry-title"><a href="http://teledramaturgia.com.br/padre-cicero/">Padre Cícero</a></h4>
http://teledramaturgia.com.br/padre-cicero/
<h4 class="entry-title"><a href="http://teledramaturgia.com.br/partido-alto/">Partido Alto</a></h4>
http://teledramaturgia.com.br/partido-alto/
<h4 class="entry-title"><a href="http://teledramaturgia.com.br/tenda-dos-milagres/">Tenda dos Milagres</a></h4>
http://teledramaturgia.com.br/tenda-dos-milagres/
<h

In [164]:
pd.DataFrame(dautores_teledra)

,Autor,Biografia,Novelas,link,Foto
0,Aguinaldo Silva,Aguinaldo Silva nasceu em 7 de junho de 1943 e...,Plantão de Polícia,http://teledramaturgia.com.br/plantao-de-policia/,http://www.teledramaturgia.com.br/wp-content/u...
1,Aguinaldo Silva,Aguinaldo Silva nasceu em 7 de junho de 1943 e...,Lampião e Maria Bonita,http://teledramaturgia.com.br/lampiao-e-maria-...,http://www.teledramaturgia.com.br/wp-content/u...
2,Aguinaldo Silva,Aguinaldo Silva nasceu em 7 de junho de 1943 e...,Bandidos da Falange,http://teledramaturgia.com.br/bandidos-da-fala...,http://www.teledramaturgia.com.br/wp-content/u...
3,Aguinaldo Silva,Aguinaldo Silva nasceu em 7 de junho de 1943 e...,Padre Cícero,http://teledramaturgia.com.br/padre-cicero/,http://www.teledramaturgia.com.br/wp-content/u...
4,Aguinaldo Silva,Aguinaldo Silva nasceu em 7 de junho de 1943 e...,Partido Alto,http://teledramaturgia.com.br/partido-alto/,http://www.teledramaturgia.com.br/wp-content/u...
...,...,...,...,...,...
767,Rubens Ewald Filho,"Nascido em Santos (SP), em 7 de março de 1945,...",Um Homem Muito Especial,http://teledramaturgia.com.br/um-homem-muito-e...,http://www.teledramaturgia.com.br/wp-content/u...
768,Rubens Ewald Filho,"Nascido em Santos (SP), em 7 de março de 1945,...",O Pátio das Donzelas,http://teledramaturgia.com.br/o-patio-das-donz...,http://www.teledramaturgia.com.br/wp-content/u...
769,Rubens Ewald Filho,"Nascido em Santos (SP), em 7 de março de 1945,...",Casa de Pensão,http://teledramaturgia.com.br/casa-de-pensao/,http://www.teledramaturgia.com.br/wp-content/u...
770,Rubens Ewald Filho,"Nascido em Santos (SP), em 7 de março de 1945,...",Iaiá Garcia,http://teledramaturgia.com.br/iaia-garcia/,http://www.teledramaturgia.com.br/wp-content/u...


In [169]:
dautores_Teletrau = pd.DataFrame(dautores_teledra)
dautores_Teletrau.to_csv('dautores_Teletrau.csv')
dautores_Teletrau

,Autor,Biografia,Novelas,link,Foto
0,Aguinaldo Silva,Aguinaldo Silva nasceu em 7 de junho de 1943 e...,Plantão de Polícia,http://teledramaturgia.com.br/plantao-de-policia/,http://www.teledramaturgia.com.br/wp-content/u...
1,Aguinaldo Silva,Aguinaldo Silva nasceu em 7 de junho de 1943 e...,Lampião e Maria Bonita,http://teledramaturgia.com.br/lampiao-e-maria-...,http://www.teledramaturgia.com.br/wp-content/u...
2,Aguinaldo Silva,Aguinaldo Silva nasceu em 7 de junho de 1943 e...,Bandidos da Falange,http://teledramaturgia.com.br/bandidos-da-fala...,http://www.teledramaturgia.com.br/wp-content/u...
3,Aguinaldo Silva,Aguinaldo Silva nasceu em 7 de junho de 1943 e...,Padre Cícero,http://teledramaturgia.com.br/padre-cicero/,http://www.teledramaturgia.com.br/wp-content/u...
4,Aguinaldo Silva,Aguinaldo Silva nasceu em 7 de junho de 1943 e...,Partido Alto,http://teledramaturgia.com.br/partido-alto/,http://www.teledramaturgia.com.br/wp-content/u...
...,...,...,...,...,...
767,Rubens Ewald Filho,"Nascido em Santos (SP), em 7 de março de 1945,...",Um Homem Muito Especial,http://teledramaturgia.com.br/um-homem-muito-e...,http://www.teledramaturgia.com.br/wp-content/u...
768,Rubens Ewald Filho,"Nascido em Santos (SP), em 7 de março de 1945,...",O Pátio das Donzelas,http://teledramaturgia.com.br/o-patio-das-donz...,http://www.teledramaturgia.com.br/wp-content/u...
769,Rubens Ewald Filho,"Nascido em Santos (SP), em 7 de março de 1945,...",Casa de Pensão,http://teledramaturgia.com.br/casa-de-pensao/,http://www.teledramaturgia.com.br/wp-content/u...
770,Rubens Ewald Filho,"Nascido em Santos (SP), em 7 de março de 1945,...",Iaiá Garcia,http://teledramaturgia.com.br/iaia-garcia/,http://www.teledramaturgia.com.br/wp-content/u...


# Tratamento do Dados

## Novelas Audiência Média

In [24]:
dnovelas_media = pd.read_csv('dnovela_media.csv')
dnovelas_media

,Unnamed: 0,0,1,2,Emissora,Horario
0,0,Ano,Novela,Média,GLOBO,EDIÇÃO ESPECIAL
1,1,2021,O Cravo e a Rosa,140,GLOBO,EDIÇÃO ESPECIAL
2,2,2022,Chocolate com Pimenta,-,GLOBO,EDIÇÃO ESPECIAL
3,0,Ano,Novela,Média,GLOBO,VALE A PENA VER DE NOVO
4,1,1980,Dona Xepa,-,GLOBO,VALE A PENA VER DE NOVO
...,...,...,...,...,...,...
2692,7,REPRISES,REPRISES,NaN,REDETV!,NOVELAS
2693,8,NaN,NaN,NaN,REDETV!,NOVELAS
2694,9,Ano,Novela,Média,REDETV!,NOVELAS
2695,10,2004,"Betty, a Feia",11,REDETV!,NOVELAS


In [37]:
dnovelas_media.columns = ['Indice','Ano','Novela','Média','Emissora','Horário']
dnovelas_media.drop(dnovelas_media[dnovelas_media['Novela'] == 'Novela'].index, inplace = True)



In [46]:
dnovelas_media.dtypes

Indice       int64
Ano         object
Novela      object
Média       object
Emissora    object
Horário     object
dtype: object

In [8]:
def fabrica_id(nome):
  nome = str(nome)
  nome = nome.replace(' ','')
  nome = nome.replace(':','')
  nome = nome.replace(';','')
  nome = nome.replace('.','')
  nome = nome.replace(',','')
  nome = nome.replace('!','')
  nome = nome.replace('-','')
  nome = nome.replace('_','')
  nome = nome.replace('@','')
  nome = nome.replace('_','')
  nome = nome.replace('&','e')
  nome = nome.replace('#','')
  nome = nome.replace('%','')
  nome = nome.replace('$','')
  nome = nome.replace('~','')
  nome = nome.replace('\n','')
  nome = nome.replace('(','')
  nome = nome.replace(')','')
  nome = nome.lower()
  nome = unidecode(nome)
  return nome


In [51]:
dnovelas_media['ID_novel'] = dnovelas_media['Novela'].apply(fabrica_id) + dnovelas_media['Ano']
dnovelas_media.to_csv('dnovela_media(final).csv')

In [ ]:
dnovelas_teledra.to_csv('/content/gdrive/My Drive/Projeto_globo/dnovelas_Teletrau12(final).csv)')

### Tratamento dnovelas_final

In [60]:
faudiencia = pd.read_csv('dnovelas_final.csv')
faudiencia

,Unnamed: 0,Novela,Autor,Semana,Dia Semana,Data,Pontos Audiencia,ano,Emissora,horario
0,0,\nO Cravo e a Rosa,NaN,Semana 02,Segunda,Semana 02: 13/12 a 17/12/2021,10,2021)\n,GLOBO,EDIÇÃO ESPECIAL
1,1,\nO Cravo e a Rosa,NaN,Semana 02,Terça,Semana 02: 13/12 a 17/12/2021,12,2021)\n,GLOBO,EDIÇÃO ESPECIAL
2,2,\nO Cravo e a Rosa,NaN,Semana 02,Quarta,Semana 02: 13/12 a 17/12/2021,12,2021)\n,GLOBO,EDIÇÃO ESPECIAL
3,3,\nO Cravo e a Rosa,NaN,Semana 02,Quinta,Semana 02: 13/12 a 17/12/2021,10,2021)\n,GLOBO,EDIÇÃO ESPECIAL
4,4,\nO Cravo e a Rosa,NaN,Semana 02,Sexta,Semana 02: 13/12 a 17/12/2021,10,2021)\n,GLOBO,EDIÇÃO ESPECIAL
...,...,...,...,...,...,...,...,...,...,...
240484,240484,"\nBetty, a Feia",FERNANDO GAITÁN,Semana 40,Segunda,Semana 40: 02/12 a 06/12/2013,01,2013)\n,REDETV!,NOVELAS
240485,240485,"\nBetty, a Feia",FERNANDO GAITÁN,Semana 40,Terça,Semana 40: 02/12 a 06/12/2013,01,2013)\n,REDETV!,NOVELAS
240486,240486,"\nBetty, a Feia",FERNANDO GAITÁN,Semana 40,Quarta,Semana 40: 02/12 a 06/12/2013,01,2013)\n,REDETV!,NOVELAS
240487,240487,"\nBetty, a Feia",FERNANDO GAITÁN,Semana 40,Quinta,Semana 40: 02/12 a 06/12/2013,01,2013)\n,REDETV!,NOVELAS


In [6]:
def limpa(coluna):
    coluna = coluna.replace('\n','')
    coluna = coluna.replace(')','')
    return coluna

In [64]:
faudiencia['Novela'] = faudiencia['Novela'].apply(limpa)
faudiencia['ano'] = faudiencia['ano'].apply(limpa)
faudiencia['ID_novel'] = faudiencia['Novela'].apply(fabrica_id) + faudiencia['ano']
faudiencia

,Unnamed: 0,Novela,Autor,Semana,Dia Semana,Data,Pontos Audiencia,ano,Emissora,horario,ID_novel
0,0,O Cravo e a Rosa,NaN,Semana 02,Segunda,Semana 02: 13/12 a 17/12/2021,10,2021,GLOBO,EDIÇÃO ESPECIAL,ocravoearosa2021
1,1,O Cravo e a Rosa,NaN,Semana 02,Terça,Semana 02: 13/12 a 17/12/2021,12,2021,GLOBO,EDIÇÃO ESPECIAL,ocravoearosa2021
2,2,O Cravo e a Rosa,NaN,Semana 02,Quarta,Semana 02: 13/12 a 17/12/2021,12,2021,GLOBO,EDIÇÃO ESPECIAL,ocravoearosa2021
3,3,O Cravo e a Rosa,NaN,Semana 02,Quinta,Semana 02: 13/12 a 17/12/2021,10,2021,GLOBO,EDIÇÃO ESPECIAL,ocravoearosa2021
4,4,O Cravo e a Rosa,NaN,Semana 02,Sexta,Semana 02: 13/12 a 17/12/2021,10,2021,GLOBO,EDIÇÃO ESPECIAL,ocravoearosa2021
...,...,...,...,...,...,...,...,...,...,...,...
240484,240484,"Betty, a Feia",FERNANDO GAITÁN,Semana 40,Segunda,Semana 40: 02/12 a 06/12/2013,01,2013,REDETV!,NOVELAS,bettyafeia2013
240485,240485,"Betty, a Feia",FERNANDO GAITÁN,Semana 40,Terça,Semana 40: 02/12 a 06/12/2013,01,2013,REDETV!,NOVELAS,bettyafeia2013
240486,240486,"Betty, a Feia",FERNANDO GAITÁN,Semana 40,Quarta,Semana 40: 02/12 a 06/12/2013,01,2013,REDETV!,NOVELAS,bettyafeia2013
240487,240487,"Betty, a Feia",FERNANDO GAITÁN,Semana 40,Quinta,Semana 40: 02/12 a 06/12/2013,01,2013,REDETV!,NOVELAS,bettyafeia2013


In [65]:
faudiencia.to_csv('faudiencia(final).csv')

### Tratamento Novelas Teletrau

In [4]:
dnovelas_Teletrau = pd.read_csv('dnovelas_Teletrau.csv')
dnovelas_Teletrau

,Unnamed: 0,Nome,Site,Sinopse,Canal,Horario,Capitulos,imagem,Ano,Elenco,Núcleos,Bastidores,Trilha Sonora,Abertura,Trilha sonora,Trama Policial,Resumos,Trama dos assassinatos
0,0,tempos modernos,http://teledramaturgia.com.br/tempos-modernos/,Leal Cordeiro (Antônio Fagundes) é um homem de...,Globo,19h,161 capítulos,http://teledramaturgia.com.br/wp-content/uploa...,2010,\nANTÔNIO FAGUNDES – Leal Cordeiro\nELIANE GIA...,\n– núcleo de LEAL CORDEIRO (Antônio Fagundes)...,\nÚnica novela de Bosco Brasil como autor titu...,\nTrilha Sonora Nacional\n\n01. 10 MINUTOS – A...,CÉREBRO ELETRÔNICO – Myllena,NaN,NaN,NaN,NaN
1,1,uma rosa com amor,http://teledramaturgia.com.br/uma-rosa-com-amo...,Serafina Rosa Petrone (Carla Marins) é uma sec...,SBT,20h,145 capítulos,http://www.teledramaturgia.com.br/wp-content/u...,2010,\n\nCARLA MARINS – Serafina Rosa PetroneCLÁUDI...,"\n\n– núcleo de SERAFINA ROSA (Carla Marins), ...","\n\nEm 2010, o SBT comprou os direitos da obra...",\n\nTrilha Sonora\n01. PRETTY WOMAN – Roy Orbi...,PRETTY WOMAN – Roy Orbinson,NaN,NaN,NaN,NaN
2,2,escrito nas estrelas,http://teledramaturgia.com.br/escrito-nas-estr...,"O Dr. Ricardo Aguillar (Humberto Martins), viú...",Globo,18h,143 capítulos,http://teledramaturgia.com.br/wp-content/uploa...,2010,\n\nHUMBERTO MARTINS – Ricardo Aguillar / Pedr...,"\n\n– núcleo de RICARDO (Humberto Martins), vi...","\n\nCom poucos tropeços e um saldo positivo, a...",\n\n\n01. QUANDO A CHUVA PASSAR – Paula Fernan...,QUANDO A CHUVA PASSAR – Paula Fernandes,NaN,NaN,NaN,NaN
3,3,passione,http://teledramaturgia.com.br/passione/,Quando Bete Gouveia (Fernanda Montenegro) conh...,Globo,21h,209 capítulos,http://www.teledramaturgia.com.br/wp-content/u...,2010,\n\nTONY RAMOS – Totó (Antonio Mattoli)MARIANA...,\n\n– núcleo de BETE GOUVEIA (Fernanda Montene...,"\n\nPassione explorou alguns temas difíceis, c...",\n\nTrilha Sonora Nacional\n\n01. AQUILO QUE D...,AQUILO QUE DÁ NO CORAÇÃO – Lenine,NaN,NaN,NaN,NaN
4,4,ribeirão do tempo,http://teledramaturgia.com.br/ribeirao-do-tempo/,O jovem empresário Tito (Ângelo Paes Leme) é n...,NaN,NaN,250 capítulos,http://www.teledramaturgia.com.br/wp-content/u...,2010,\n\nBIANCA RINALDI – ArmindaCAIO JUNQUEIRA – J...,"\n\n– núcleo de ARMINDA (Bianca Rinaldi), pode...","\n\nPara escrever Ribeirão do Tempo, o autor, ...",\n\nTrilha Sonora\n01. FÃ – Christian & Cristi...,PIQUE DO TEMPO – Tom Zé,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,694,dez vidas,http://teledramaturgia.com.br/dez-vidas/,"A Inconfidência Mineira é retratada, tendo com...",NaN,NaN,neiro de 1970,http://www.teledramaturgia.com.br/wp-content/u...,1969,\nCARLOS ZARA – Tiradentes (Joaquim José da Si...,NaN,\nÚltima das treze novelas consecutivas que Iv...,\nTrilha Sonora: Manuel Marques\n\nTEMA DE TIR...,NaN,NaN,NaN,NaN,NaN
695,695,seu único pecado,http://teledramaturgia.com.br/seu-unico-pecado/,Um jovem vem para São Paulo à procura de seu t...,Record,19h,embro de 1969,http://www.teledramaturgia.com.br/wp-content/u...,1969,\nADEMIR ROCHA\nSUSANA VIEIRA\nÍRIS BRUZZI\nRO...,NaN,\nPercebe-se na trama a inspiração no livro de...,NaN,NaN,NaN,NaN,NaN,NaN
696,696,véu de noiva,http://teledramaturgia.com.br/veu-de-noiva/,"Andreia, moça simples e generosa, de família h...",Globo,20h,21 capítulos,http://teledramaturgia.com.br/wp-content/uploa...,1969,\n\nREGINA DUARTE – Andreia Madeira / Roberta ...,"\n\n– núcleo de ANDREIA (Regina Duarte), jovem...",\nInfluenciada pelo sucesso da novela Beto Roc...,\n\nTrilha Sonora\n\n01. TEMA DE LUCIANO – Lui...,NaN,NaN,NaN,NaN,NaN
697,697,super plá,http://teledramaturgia.com.br/super-pla/,"Plácido, quando criança, era super inteligente...",NaN,NaN,maio de 1970,http://www.teledramaturgia.com.br/wp-content/u...,1969,\nRODRIGO SANTIAGO – Plácido\nBETE MENDES – Ti...,NaN,"\nApós o sucesso de Beto Rockfeller, o autor B...",\nTrilha Sonora 1\n\n01. JAM UP JELLY TIGHT – ...,NaN,NaN,NaN,NaN,NaN


In [10]:
dnovelas_Teletrau['Nome'] = dnovelas_Teletrau['Nome'].apply(limpa)
dnovelas_Teletrau['Ano'] = dnovelas_Teletrau['Ano'].apply(limpa)
dnovelas_Teletrau['ID_novel'] = dnovelas_Teletrau['Nome'].apply(fabrica_id) + dnovelas_Teletrau['Ano']
dnovelas_Teletrau.to_csv('dnovelas_Teletrau(final).csv')
dnovelas_Teletrau

,Unnamed: 0,Nome,Site,Sinopse,Canal,Horario,Capitulos,imagem,Ano,Elenco,Núcleos,Bastidores,Trilha Sonora,Abertura,Trilha sonora,Trama Policial,Resumos,Trama dos assassinatos,ID_novel
0,0,tempos modernos,http://teledramaturgia.com.br/tempos-modernos/,Leal Cordeiro (Antônio Fagundes) é um homem de...,Globo,19h,161 capítulos,http://teledramaturgia.com.br/wp-content/uploa...,2010,\nANTÔNIO FAGUNDES – Leal Cordeiro\nELIANE GIA...,\n– núcleo de LEAL CORDEIRO (Antônio Fagundes)...,\nÚnica novela de Bosco Brasil como autor titu...,\nTrilha Sonora Nacional\n\n01. 10 MINUTOS – A...,CÉREBRO ELETRÔNICO – Myllena,NaN,NaN,NaN,NaN,temposmodernos2010
1,1,uma rosa com amor,http://teledramaturgia.com.br/uma-rosa-com-amo...,Serafina Rosa Petrone (Carla Marins) é uma sec...,SBT,20h,145 capítulos,http://www.teledramaturgia.com.br/wp-content/u...,2010,\n\nCARLA MARINS – Serafina Rosa PetroneCLÁUDI...,"\n\n– núcleo de SERAFINA ROSA (Carla Marins), ...","\n\nEm 2010, o SBT comprou os direitos da obra...",\n\nTrilha Sonora\n01. PRETTY WOMAN – Roy Orbi...,PRETTY WOMAN – Roy Orbinson,NaN,NaN,NaN,NaN,umarosacomamor2010
2,2,escrito nas estrelas,http://teledramaturgia.com.br/escrito-nas-estr...,"O Dr. Ricardo Aguillar (Humberto Martins), viú...",Globo,18h,143 capítulos,http://teledramaturgia.com.br/wp-content/uploa...,2010,\n\nHUMBERTO MARTINS – Ricardo Aguillar / Pedr...,"\n\n– núcleo de RICARDO (Humberto Martins), vi...","\n\nCom poucos tropeços e um saldo positivo, a...",\n\n\n01. QUANDO A CHUVA PASSAR – Paula Fernan...,QUANDO A CHUVA PASSAR – Paula Fernandes,NaN,NaN,NaN,NaN,escritonasestrelas2010
3,3,passione,http://teledramaturgia.com.br/passione/,Quando Bete Gouveia (Fernanda Montenegro) conh...,Globo,21h,209 capítulos,http://www.teledramaturgia.com.br/wp-content/u...,2010,\n\nTONY RAMOS – Totó (Antonio Mattoli)MARIANA...,\n\n– núcleo de BETE GOUVEIA (Fernanda Montene...,"\n\nPassione explorou alguns temas difíceis, c...",\n\nTrilha Sonora Nacional\n\n01. AQUILO QUE D...,AQUILO QUE DÁ NO CORAÇÃO – Lenine,NaN,NaN,NaN,NaN,passione2010
4,4,ribeirão do tempo,http://teledramaturgia.com.br/ribeirao-do-tempo/,O jovem empresário Tito (Ângelo Paes Leme) é n...,NaN,NaN,250 capítulos,http://www.teledramaturgia.com.br/wp-content/u...,2010,\n\nBIANCA RINALDI – ArmindaCAIO JUNQUEIRA – J...,"\n\n– núcleo de ARMINDA (Bianca Rinaldi), pode...","\n\nPara escrever Ribeirão do Tempo, o autor, ...",\n\nTrilha Sonora\n01. FÃ – Christian & Cristi...,PIQUE DO TEMPO – Tom Zé,NaN,NaN,NaN,NaN,ribeiraodotempo2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,694,dez vidas,http://teledramaturgia.com.br/dez-vidas/,"A Inconfidência Mineira é retratada, tendo com...",NaN,NaN,neiro de 1970,http://www.teledramaturgia.com.br/wp-content/u...,1969,\nCARLOS ZARA – Tiradentes (Joaquim José da Si...,NaN,\nÚltima das treze novelas consecutivas que Iv...,\nTrilha Sonora: Manuel Marques\n\nTEMA DE TIR...,NaN,NaN,NaN,NaN,NaN,dezvidas1969
695,695,seu único pecado,http://teledramaturgia.com.br/seu-unico-pecado/,Um jovem vem para São Paulo à procura de seu t...,Record,19h,embro de 1969,http://www.teledramaturgia.com.br/wp-content/u...,1969,\nADEMIR ROCHA\nSUSANA VIEIRA\nÍRIS BRUZZI\nRO...,NaN,\nPercebe-se na trama a inspiração no livro de...,NaN,NaN,NaN,NaN,NaN,NaN,seuunicopecado1969
696,696,véu de noiva,http://teledramaturgia.com.br/veu-de-noiva/,"Andreia, moça simples e generosa, de família h...",Globo,20h,21 capítulos,http://teledramaturgia.com.br/wp-content/uploa...,1969,\n\nREGINA DUARTE – Andreia Madeira / Roberta ...,"\n\n– núcleo de ANDREIA (Regina Duarte), jovem...",\nInfluenciada pelo sucesso da novela Beto Roc...,\n\nTrilha Sonora\n\n01. TEMA DE LUCIANO – Lui...,NaN,NaN,NaN,NaN,NaN,veudenoiva1969
697,697,super plá,http://teledramaturgia.com.br/super-pla/,"Plácido, quando criança, era super inteligente...",NaN,NaN,maio de 1970,http://www.teledramaturgia.com.br/wp-content/u...,1969,\nRODRIGO SANTIAGO – Plácido\nBETE MENDES – Ti...,NaN,"\nApós o suces

### Tratamento Novelas Teletrau

In [76]:
dautores_Teletrau = pd.read_csv('dautores_Teletrau.csv')
dautores_Teletrau.to_csv('dautores_Teletrau(final).csv')